In [ ]:
# --- Загружаем модули ------------------------------------
import numpy as np # линейка
import pandas as pd # обработка данных, загрузка CSV

from sklearn.grid_search import GridSearchCV

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split    
from sklearn.decomposition import PCA # размерности 
from sklearn.model_selection import KFold

import lightgbm as lgb
from subprocess import check_output
import re

In [ ]:
K = 10
MAX_ROUNDS = 2500
Oxy = 1
Clear = 1
Electro = 1

In [ ]:
def Floated(DS):
    DS.Ga_0 = DS.Ga_0.astype("float"); DS.Ga_1 = DS.Ga_1.astype("float"); DS.Ga_2 = DS.Ga_2.astype("float"); DS.Ga_3 = DS.Ga_3.astype("float"); DS.Ga_4 = DS.Ga_4.astype("float"); DS.Ga_5 = DS.Ga_5.astype("float")
    DS.Al_0 = DS.Al_0.astype("float"); DS.Al_1 = DS.Al_1.astype("float"); DS.Al_2 = DS.Al_2.astype("float"); DS.Al_3 = DS.Al_3.astype("float"); DS.Al_4 = DS.Al_4.astype("float"); DS.Al_5 = DS.Al_5.astype("float")
    DS.In_0 = DS.In_0.astype("float"); DS.In_1 = DS.In_1.astype("float"); DS.In_2 = DS.In_2.astype("float"); DS.In_3 = DS.In_3.astype("float"); DS.In_4 = DS.In_4.astype("float"); DS.In_5 = DS.In_5.astype("float")
    DS.O_0 = DS.O_0.astype("float");   DS.O_1 = DS.O_1.astype("float");   DS.O_2 = DS.O_2.astype("float");   DS.O_3 = DS.O_3.astype("float");   DS.O_4 = DS.O_4.astype("float");   DS.O_5 = DS.O_5.astype("float")

In [ ]:
# --- функции -------------------------
train0 = pd.read_csv("../input/train.csv"); test0 = pd.read_csv("../input/test.csv")

In [ ]:
def get_xyz_data(filename):
    pos_data = []; lat_data = []
    with open(filename) as f:
        for line in f.readlines():
            x = line.split()
            if x[0] == 'atom':
                pos_data.append([np.array(x[1:4], dtype=np.float),x[4]])
    return pos_data

In [ ]:
def CreateCols(X_cols,Name):
    for i in range(6):
        X_cols.append(Name+str(i))

In [ ]:
def Append2List(li, X_list, Name):
        try:
            if li[1] == Name:
                X_list.append(li[0])
        except:
            pass

In [ ]:
def PCA_Fit(i, X_list, X_cols):
    try:
        model = PCA(n_components=2)
        X_list = np.array(X_list)
        temp_X = model.fit_transform(X_list.transpose())
        temp_X = [item for sublist in temp_X for item in sublist]
    except:
        temp_X = [0,0,0,0,0,0]
    temp_X = pd.DataFrame(temp_X).transpose()
    temp_X.columns = X_cols
    temp_X.index = np.array([i])
    return temp_X

In [ ]:
def LoadData(DATA_F, Name):
    ga_df = pd.DataFrame(columns = ga_cols); al_df = pd.DataFrame(columns = al_cols); in_df = pd.DataFrame(columns = in_cols); o_df  = pd.DataFrame(columns = o_cols)
    for i in DATA_F.id.values:
        fn = "../input/"+Name+"/{}/geometry.xyz".format(i)
        xyz = get_xyz_data(fn)
        ga_list = []; al_list = []; in_list = []; o_list = []
        for li in xyz:
            Append2List(li, ga_list, "Ga");  Append2List(li, al_list, "Al"); Append2List(li, in_list, "In"); Append2List(li, o_list,  "O")

        ga_df = pd.concat([ga_df, PCA_Fit(i, ga_list, ga_cols)]); al_df = pd.concat([al_df, PCA_Fit(i, al_list, al_cols)]); in_df = pd.concat([in_df, PCA_Fit(i, in_list, in_cols)]); o_df  = pd.concat([o_df,  PCA_Fit(i, o_list,  o_cols)])
    
    ga_df["id"] = ga_df.index; al_df["id"] = al_df.index; in_df["id"] = in_df.index; o_df["id"]  = o_df.index

    DATA_F = pd.merge(DATA_F, ga_df, on = ["id"], how = "left"); DATA_F = pd.merge(DATA_F, al_df, on = ["id"], how = "left"); DATA_F = pd.merge(DATA_F, o_df,  on = ["id"], how = "left"); DATA_F = pd.merge(DATA_F, in_df, on = ["id"], how = "left")
    Floated(DATA_F)
    if Oxy == 1:
        DATA_F['number_of_oxy'] = DATA_F['number_of_total_atoms'] * 0.6
    if Electro == 1:
        DATA_F['Electro_Ga'] = DATA_F['percent_atom_ga']      * 0.4 * 1.81
        DATA_F['Electro_Al'] = DATA_F['percent_atom_al']      * 0.4 * 1.61
        DATA_F['Electro_In'] = DATA_F['percent_atom_in']      * 0.4 * 1.78
        DATA_F['Electro_O'] = DATA_F['number_of_total_atoms'] * 0.6 * 3.44
    return DATA_F

In [ ]:
def Clear_(dtrain, dtest):    
    all_df = pd.concat([dtrain, dtest], ignore_index=True)
    all_df["directory"] = ['test' if np.isnan(bg) else 'train' for bg in all_df.bandgap_energy_ev.values]
    geom_data = []
    for row in all_df.itertuples():
        File='../input/' + row.directory + '/{}/geometry.xyz'.format(row.id)
        with open(File) as f:
            text = f.read()
            geom_data.append(text)
    
    same_geoms = []
    TotLen=len(dtrain)
    for i in range(len(geom_data)):
        for j in range(i):
            if geom_data[i] == geom_data[j]:
                if i < TotLen:
                    same_geoms.append(i)
                if j < TotLen:
                    same_geoms.append(j)
    same_geoms.sort()
    dtrain = dtrain.drop(same_geoms)
    dtrain = dtrain.reset_index(drop=True)
    return dtrain, dtest

In [ ]:
ga_cols = []; al_cols = []; in_cols = []; o_cols  = []
CreateCols(ga_cols, "Ga_"); CreateCols(al_cols, "Al_"); CreateCols(in_cols, "In_"); CreateCols(o_cols,  "O_")
train = LoadData(train0, "train");
train['formation_energy_ev_natom'] = np.log1p(train['formation_energy_ev_natom'].values)      
train['bandgap_energy_ev']         = np.log1p(train['bandgap_energy_ev'].values)      
test  = LoadData(test0,  "test")
if Clear == 1:
    train, test = Clear_(train, test)

In [ ]:
def Train(params,X_train,y_train,y_val):
    cate_vars = []
    dtrain = lgb.Dataset(X_train.drop("id", axis = 1), label=y_train, categorical_feature=cate_vars)
    dval = lgb.Dataset(X_val.drop("id", axis = 1), label=y_val, reference=dtrain, categorical_feature=cate_vars)
    bst = lgb.train(params, dtrain, num_boost_round=MAX_ROUNDS, valid_sets=[dtrain, dval], early_stopping_rounds=50, verbose_eval=100)
    print("\n".join(("%s: %.2f" % x) for x in sorted(zip(X_train.columns, bst.feature_importance("gain")), key=lambda x: x[1], reverse=True)))
    return bst

In [ ]:
def Predict(bst):
    val_pred = []; test_pred = []
    val_pred.append(bst.predict(X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(test.drop("id",axis =1), num_iteration=bst.best_iteration or MAX_ROUNDS))
    return test_pred

In [ ]:
def rmsle(h, y): 
    return np.sqrt(np.square(np.log1p(h) - np.log1p(y)).mean())

In [ ]:
kf = KFold(n_splits = K, random_state = 3228, shuffle = True)

In [ ]:
#train.iloc[train_index].count()

In [ ]:
cnt=0
for train_index, test_index in kf.split(train):
    X_train,  X_val  = train.iloc[train_index], train.iloc[test_index]

    y_train_1 = X_train['formation_energy_ev_natom']
    y_train_2 = X_train["bandgap_energy_ev"]

    y_val_1 = X_val['formation_energy_ev_natom']
    y_val_2 = X_val["bandgap_energy_ev"]

    X_train = X_train.drop(["formation_energy_ev_natom","bandgap_energy_ev"],axis =1)
    X_val = X_val.drop(["formation_energy_ev_natom","bandgap_energy_ev"],axis =1)

    params = {
        'num_leaves': 7,
        'objective': 'regression',
        'min_data_in_leaf': 18,
        'learning_rate': 0.05,
        'feature_fraction': 0.93,
        'bagging_fraction': 0.93,
        'bagging_freq': 1,
        'metric': 'l2',
        'num_threads': 1
    }
    Model1 = Train(params, X_train, y_train_1, y_val_1)

    params = {
        'num_leaves': 8,
        'objective': 'regression',
        'min_data_in_leaf': 18,
        'learning_rate': 0.035,
        'feature_fraction': 0.95,
        'bagging_fraction': 0.95,
        'bagging_freq': 1,
        'metric': 'l2',
        'num_threads': 1
    }
    Model2 = Train(params, X_train, y_train_2, y_val_2)
    pred1 = Predict(Model1)
    pred2 = Predict(Model2)
    if cnt == 0:
        test_pred_1 = pred1[0] / K
        test_pred_2 = pred2[0] / K
    else:
        test_pred_1 = test_pred_1 + pred1[0] / K
        test_pred_2 = test_pred_2 + pred2[0] / K
    cnt+=1


In [ ]:
sample = pd.read_csv("../input/sample_submission.csv")
sample["formation_energy_ev_natom"] = np.expm1(test_pred_1)
#np.expm1(pred1_)
sample["bandgap_energy_ev"] = np.expm1(test_pred_2)
sample.to_csv("submission.csv",index = False)

In [ ]:
sample